# 텍스트 마이닝 프로젝트 (2020.6~)
### Data Crawling
#### 휴먼지능정보공학과 류지혜, 이지민

## 에어팟 1세대

In [2]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','사용성','조작성','디자인','착용감','휴대성']

#에어팟 1세대 검색 
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=10776906666&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25201%25EC%2584%25B8%25EB%258C%2580&NaPm=ct%3Dkb3ef8sg%7Cci%3Df7d5699aba286396ee3757965561aa228125c722%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Da9cfdf21495248f435f4ae3cbea263c694784ae1"
#xpath
shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[9]/a/span" #사용성
category7="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[11]/a/span" #조작성
#향상되지않은기능
category8="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[5]/a/span" #디자인
category9="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[10]/a/span" #착용감
category10="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[14]/a/span" #휴대성

header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/jmlee/Downloads/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

def save():
    if not os.path.exists('output1.csv'):
        df1.to_csv('output1.csv', encoding='utf-8-sig', mode='w')
    else:
        df1.to_csv('output1.csv',encoding='utf-8-sig', mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"성능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"기능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"배터리 수명" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"사용성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            


up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()
            
            
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()
            
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

페이지 1 

1 2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요~예쁘고 착용감이 좋아요>_< 5 

2 배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 이상 없이 잘 됐어요!~연결도 잘되고, 음악 잠깐 들었는데 음질도 좋고 정말 편해요^^미리 구입해둔 케이스도 장착해서 바로 사용했어요~가격도 L.POINT랑 청구할인 받아서 직구보다 저렴하게 잘 구입했습니다.좋은 상품 감사합니다:) 5 

3 아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아이패드로 바꿔서 들을때 불편해서 블루투스 이어폰을 검색하다 주변의 구매자들 모두 하나같이 최고다라고 이야기 한 애플의 에어팟을 구매하였습니다.

할인쿠폰과 카드청구할인 혜택으로 저렴하게 구입할 수 있었고 배송도 공휴일이 포함되었음에도 2일만에 받을 수 있어 좋았습니다.

제품은 박스를 에어캡으로 감싸고 다시 큰 박스에 넣어 안전하게 배송이 되었습니다. 제품도 2018년 5월 제조 최신 제품으로 받았습니다.

받자마자 아이폰에 연결하고 사용해봤는데 . 대단하 만족합니다. 5 

4 다른 저렴한 곳도 있었습니다.

그래도 제조일자 최신에 페이백으로 다시 받는 금액 생각해보니 얼추 괜찮다고 생각해서 구매했습니다.

사용해보니 정말 편합니다. 음질도 나쁘지 않구요.
귀에 꼽고 장시간 있으면 혹시 떨어졌나 하는 생각도 들구요 가끔씩 귀 한번 만집니다 ㅋㅋㅋ
적응하면 신경 안쓸듯합니다.너무 불안하면 실리콘 줄 하나 구매해서 쓰려고 합니다 4 

5 택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사했어요
친구들 말만듣고 좋구나...생각만 했는데
막상 사용하니 너무 좋아요 어제 노래들으면서 청소 했는데 삶의 질이 달라졌다는 말이 생각 나더라구요 ㅎㅎ 너무 마음에 들어요 그리고 차에서 선이 없

35 계속 고민하다가 사야겠다 싶어서... 엘페이 등록하고 신한카드 청구할인으로 퀵배송으로 5천원 쓰고 받았습니다. 강남이어서 그런것인지 주문하고 30분 만에 왔어요! 완전 기분 짱이네요 ㅎㅎ 음질 좋고 간편한 연결에 짱입니다. 왜 이렇게 많이들 쓰는지 알겠네요. 5 

36 처음에 출시됐을때는 뭐야? 하는 반응으로 거들떠 보지도 않았는데
아이폰8로 바꾸면서, 이어폰 구멍이 없어져서, 알아보게 됐네요~
가격이 좀 높은편이라 몇일 고민하다가, 그래도 기존 이어폰은
불편할거같아서 구매했는데, 생각보다 귀에서 떨어질 염려도 없는거같고
통화나, 음악 음질도 좋아요!
단점은, 블루투스로 해야된다는점, ^^ 4 

37 이어폰도 분실했고 차량용 블루투스도 불편해서 겸사겸사 비싼 이걸 주문했어요 사용해보니 확실히 편하긴하더라구요 귀에도 밀착감있게 착 붙어서 쉽게 떨어질듯하진 않고. 무엇보다 이어팟을 통해 듣는 소리가 되게 선명합니다 아이폰4를 처음 만났을때의 그 신선함을 오랜만에 다시 느꼈네요. 다시 만난 신세계!!상품은 되게 만족스럽지만 배송은 완전 실망!! 2일에 주문해서 오늘 배송받았고 배송지연에 대한 문의는 판매자분은 쌩 까시더라구요 기다리다 목빠지는줄!주문하고 기다리시는분은 참을 인자를 마음에 새기고 기다리셔야될듯요~기다리시면 신세계를 만나실겁니다 3 

38 에어팟 사고 싶어서 해외직구 하려다 안전하게 국내 당일배송해주셔서 바로 샀네요 여길 왜 이제 봤는지 모르겠네요 ㅠㅠ 여러분 강추 입니다 사자말자 다음날 점심에 바로 왔어요!! 스크래치도 없고 소리도 완벽 그리고 파정입니다~~ 5 

39 에어팟 비싸서 못사고 있다가 최저가 뜬거 보고 바로 구매했습니다ㅋㅋ포장도 안뜯겨 있는 완전 새거네요오자마자 사진 찍고 바로 뜯어서 제 아이폰이랑 연결했는데 너무 잘됩니다!역시 애플 이어폰이네요ㅋㅋ 음질 최고에요><선이 없으니까 걸리적거리는것도 없고 진짜 편합니다너무 마음에 들어서 지인한테도 추천했어요ㅋㅋㅋ 5 

40 콩나물 닮았다고 안살려고 했는데,평가가 좋아서 구매했는데

79 해외배송으로 오는 다른 곳에서 주문했다가 물건 주문이 무기한 연기가 되서 11번가로 찾았는데 배송도 엄청 빠르고 제품도 2017년 제조 물건이고 안전하게 잘 왔습니다! 돈값을 하는거 같이 편리하기도 하고 음질도 좋네요!! 굿굿굿 5 

80 이어폰 줄때문에 스트레스 받아서 구입한건데 제 폰이 문제인건지...지하철에서 음악듣는데 노이즈가 너무 심합니다 전화할때도 유선이어폰과 달리 마이크가 에어팟 자체에 붙어있어서 듣는사람이 주변소음이 너무 크게들린다고 하네요 3 

페이지 5 

81 배송짱빠름 블투이어폰 사려는데 이것도 뭔가 맘에안들고 저것도 뭔가 맘에안들고,, 이러다가 사는김에 그냥 크게 지르자 해서 에어팟구매 ! 11번가 무이자할부덕에 사는김에 애플워치도 같이 구매했음 원래 애플정품 이어폰보다 저음이 좀 빵빵해진거같고 블루투스이어폰 처음쓰는데 에어팟 진짜 짱인듯.. 5 

82 아이폰 사용자여서 이어폰잭이 없어 충전과 이어팟을 같이 못 써서 매우 불편해 큰 맘 먹고 샀네요. 이어팟 줄 풀 때마다 가위로 자르고 싶었는데 에어팟은 신세계네요 ㅎㅎㅎ 다만 성능 대비 가성비가 너무 떨어지고 음질도 그닥 좋지도 않네요… 편의성은 단연 모든 블루투스 이어폰 중 최고입니다. 돈이 넘치는 분에게 추천합니다. 3 

83 일단 배송이 상당히 빨리와서 좋습니다 ㅋㅋ블루투스 이어폰도 처음인데 무선이라 어색하긴 하지만 상당히 편합니다. 음색도 이어팟보다 좀 더 선명해진 느낌입니다. 더 써봐야 알겠지만 오늘 받아서 써 본 느낌은 아주 만족스럽습니다. 애플 유저분들은 후회하지 않을 아이템이네요~ 5 

84 배송짱빠르고 드디어 줄을 안풀고 들어도 된다는게 감동이네요 ㅜㅠ 음질도 좋고 역시 에어팟!’ 신세계네요. 5 

85 음질도 좋구 빠른배송 아주 좋았어요 5 

86 안드로이드에서 1세대가 음악이 덜 끊긴다고 해서 시킨건데 2세대가 왔어요; 사람 많은곳이나 버스, 지하철 지나가면 음악 듣기 힘들어요; 음질은 좋아요; 유선하고 병행해서 써야할듯 싶습니다 2 

87 배송도 

228 무선이라 무시했었는데 음질도좋고 역시에플 5 

229 적당한 배송과 적당한 가격에 구매한거 같습니다. 요즘 카피제품 많이 판매되는거 같던데 역시 정품이 좋은거 같네요. 음질 좋습니다. 3 

230 배송도 빠르고 음질도 좋아요!! 4 

231 음질좋고 괜찮습니다. 5 

232 생각보다 음질이 좋습니다. 이어팟 비교 중저음이 좋습니다. 구매하실분 참고 하세요~~ 4 

233 안드로이드 폰을 사용하지만 너무 쓰고싶어서 남편꺼랑 2개 주문했어요~ 잘 되고 정품등록도 됫어요 좋네요!음질도 좋고 싸게 살수 있어서 좋아요 5 

234 귀가 커서 조금 안맞긴 하지만 기능상으로나 너무 마음에 들어요 음질도 좋고 그리고 되게 편리해요 5 

235 음질도 좋고 통화음질도 좋네요.굿입니다.너무 맘에들어요. 적극추천합니다. 5 

236 진짜 너무 맘에 드네요. 음질도 좋고,제조일도 맘에 드네요.특히 배송이 빠름!오후2시쯤 구매했는데 다음날 오후 한시에 받았어요 울산인데대박! 5 

237 음악 감상이나 통화시 음질이 좋고 마음에 듭니다. 5 

238 배송도 빠르고 음질도 좋고 짱이에요!! 5 

239 음질도 좋고 포장상태도 좋네요😍 5 

240 배송도 빠르고 음질도 좋아요 5 

페이지 13 

241 음질 좋아요.배송도 빨라요 5 

242 갖고싶었는데이제야 갖게됐어요 ㅎㅎㅎ너무 예뻐요음질도좋구 5 

243 이걸 왜 이제야 샀을까 후회했네요 연결도 편리하고 음질도 좋아요 배송도 빠르구요 신세계 경험중 5 

244 무건이어폰 싼거써봤는데 페어링 끊김현상때문에 짜증납니다비싸도 품질 음질 서비스 3박자가 맞는게 좋아요 5 

245 음질 선명하고 상태 좋습니다 감사합니다! 5 

246 많은 사람들이 이제품을 귀에다 걸고 다니는지 더 자세히 알것같습니다. 충전이나 사용할때의 편의성이 탁월하며 음질 또한 좋습니다. 이건 무조건 사야됩니다 5 

247 운동할때 너무 편해서 왜 진작 사지 않았을까 싶네요!!! 사운드도 이어팟보다 더 좋은 것 같아욤^^ 5 



357 깔끔하게 잘 보내주셔서 감사합니다 원래 이어팟 쓰던사람이라 역시 에어팟도 좋네요 음질도 괜찮고 성능도 문제없어요~ 타 사이트보다 싸게 구매할수 있어서도 좋았습니다 5 

358 배송도 완전 빠르고 음질도 완전 좋아요ㅋㅋㅋ완전 대만족ㅋㅋㅋ 완전 좋아욬ㅋㅋㅋ 재구매 의사율도 아주 높습니다 5 

359 빠른 배송~ 선이 없어서 무척 편합니다. 가격이 비싸다는게 단점이라면 단점이죠. 핸드폰 하고 거리가3m이내 벽이 하나 있으면 소리가 끊겨요~ㅎ (블루투스의 숙명??ㅋ) 여튼 편하고 좋네요. 4 

360 가격때문에 고민 많이 했는데
타 싸이트보다 저렴하고 무엇보다 당일 주문해서 당일날 받을 수 있어서 너무 좋아요~
생각보다 음질도 괜찮고 너무 좋네요~
안잊어버리고 잘써야겠어요^^ 5 

페이지 19 

361 설명서가 불친절한건 애플 사용자면 다 아시겠고, 음질이 생각보다 엄청 좋아서 놀랬네요. 5 

362 음질이 좋긴 좋네요근데 내귀가 짝짝인가 왼쪽귀가 꽉 고정이 안되는건 왜그러지ㅠ오르쪽은 딱 좋은데요 헐 4 

363 역시 너무 편리해요.
음질도 소름끼치게 좋고.
포장도 튼튼하게 잘 왔어용!! 5 

364 가격이 다소 높지만 편리성 음질 괜찮네요- 운동할 때 특히 좋아요~ 5 

365 그저그래요 음질은 그냥 이어팟이고소리가 자주 끊겨서 스트레스 3 

366 완전 조아요 ㅎㅎ 음질도 좋고 이어팟이랑 착용감도 비슷해요 5 

367 음질이 너무 좋네요! 잃어버리지만 않으면 되겠습니다 5 

368 배송은 일주일넘게 걸렸어요. 음질은 좋아요 4 

369 받자마자 잘사용중!!
음질도 좋고 편하네요~ 5 

370 잘 받았습니다.. 좋아요.. 소리도 선명하고 5 

371 정말 좋습니다 ㅜㅠㅠ 진짜 신세계네요 ㅜㅠㅠ 완전 좋아요 인식도 잘되고 연결하는것도 간편해서 좋고 음질도 좋습니당ㅎㅎ 만족해요 많이파세요~~ 5 

372 왜 에어팟에어팟 하는지 알겠어요 솔직히 너무좋아요

귀에서 잘 빠지지도 않고 가볍고 음질도 좋고 통화도 잘되고 !!추천합니다 구매하

...

20 성능이좋대요배송도빨라요ㅎ 5 

페이지 2 

21 성능 짱 좋습니다. 5 

22 배송 정말 빨라요..! 제품도 잘 받았고요!질문답변에서 보고 정품인거 믿고 주문했습니다. 블루투스 연결도 빠르고 첫 무선 이어폰 구매인데 조심히 잘 사용해 볼게요. 다른곳보다 더 저렴하게 살수있어서 너무 좋았어요! 5 

23 최저가로 구매해서 좋아요! 음질 성능 단연 최고입니다^^ 5 

24 디자인 심플, 성능 굿굿.. 5 

25 성능이 너무 좋아요 기대이상입니다 5 

26 배송도 빠르고 성능도 너무 좋고 이쁩니다😍😍 5 

27 기대보다 성능좋네요 콩나물도 귀엽고 최근제조 마음에 들어요 다만 왼쪽에 마감이 미세하게 석연찮지만 제가 쓸고라 그냥 씁니다 5 

28 배송도 빠르고 성능도 좋고 이쁩니다. 음질도 깨끗하고 색상도 좋고 케이스도 깜찍하니 매우 만족합니다. 5 

29 성능좋아요 잘쓰고있어요 5 

30 할인잘받아서 저렴하게 샀습니다. 성능이나 편의성도 매우좋아요. 매우 만족합니다. 5 

31 내년 3월에 2세대 나올때까지 존버할까 살까 고민하던 찰나에 특가에 풀려서 운좋게 겟했네여. 성능은 뭐 말할 것도 없이 좋네요. 배송도 하루만에 오고 완전 추천합니다![G마켓 안드로이드 앱에서 작성] 5 

32 음량도 좋고 인식도 빠르며 매우깔끔해 좋습니다! 좋은 상품 감사드립니다! 5 

33 아주 귀엽고 아담하고 편해서 좋아요. 배송비가 좀 비싸서 망설였지만 너무 예쁘고 성능도 좋아서 만족합니다. 5 

34 아주 좋아요
꼭 구매 하세요
성능 좋아요 5 

35 예쁘고 성능좋고 만족이요 5 

36 인식도 빠르고 깔끔하니 좋습니다.단, 구매 전 한번 착용해보고 살 것을 추천드립니다.저 같은 경우는 귀에 잘 안 맞아서.. 귀에서 잘 빠지네요. 4 

37 앙증맞고 귀엽네요. 성능 굿. 5 

38 너무 좋아요!! 배송도 빠르고 성능도 좋아요! 5 

39 성능이나 기능은 말할것도 없이 너무 좋네요ㅎㅎ 5 

40 배송, 포장, 성능 모두 굿입니다 5 

페이지 3

8 아주 빠르게 배송받았습니다.아이폰X 쓰는데 에어팟을 열기만했는데 자동으로 페어링이 되네요본체배터리 좌우 에어팟 각각 배터리 잔량이 정말 예쁘게 뜨는데 이래서 에어팟 에어팟하나 싶네요1년전에 사지않은걸 후회합니다. 5 

9 아주 빠르게 배송받았습니다.아이폰X 쓰는데 에어팟을 열기만했는데 자동으로 페어링이 되네요본체배터리 좌우 에어팟 각각 배터리 잔량이 정말 예쁘게 뜨는데 이래서 에어팟 에어팟하나 싶네요1년전에 사지않은걸 후회합니다. 5 

10 쇼킹딜에 뜨는 싼 에어팟 사지 마시고 여기서 사세요!!!!! 거기서 구매한 리뷰자분들의 페어링하면 이미 ㅇㅇ의 Airpods 같이 다른사람 이름으로 페어링된대서 가격이 조금 있어도 믿고 구매했어요. 저는 페어링하니까 바로 제 아이폰 이름으로 제이름의 에어팟 이렇게 떴습니다!! 빨리 받고싶어 퀵으로 거래했고 너무 친절하게 응대해주셔서 주문한지 4시간도 안되어 받았습니다!! 작동 잘 되고 제조일자도 확인되고 뽁뽁이도 정말 안전하게 해주시고 감사합니당!! 단골해놓고 다음에 또 구매할 의사 있어요!!! 리뷰 진짜 안쓰는데 사장님 너무 좋으셔서 ㅠㅠㅠ 쓰고갑니다!!! 번창하세요! 5 

11 하이마트 특가이벤트로 장만했는데 하루만에 배송받아서 바로 사용할 수 있어서 정말 좋습니다 기존 이어폰 줄때문에 너무 번거로워서 별 기대없이 샀는데 사운드도 좋구요 케이스도 그냥 케이스가 아니더라구요 ㅎㅎ 착용하면 바로 자동으로 인식되는 기능도 좋아요^^ 5 

12 음질 : 맥 프레 2017년 형 페어링 시에 이상하게 주변에 이전 연결한 아이폰이 있으면 가끔 끊기면서 들릴 때가 있습니다. 이유는 명확히 모르겠네요; 배송 : 보통입니다. 평가 : 우선은 맥이나 패드 아이폰 같이 쓰는 분들은 멀티 페어링이 안되니 주의하세요. 신박하게 슉슉 하면서 페어링 되지 않습니다.; 3 

13 맘에 들어요^^ 너무 좋은 가격이라 급하게 구매했는데 생각했던 것보다 훨씬 음질도 좋고 페어링도 잘 돼서 정말 만족스러워요. 혹시나 귀에 안 맞을까도 걱정이었는

4 광고로만 보던제품을 직접사용해보니 넘 좋더라구요.간편한건 당연~~~음악을 들으니 콘서트에 온것같아요.화장하면서도 줄이 꼬이지않아 편리하구 잠잘때 asmr을 즐겨듣는데 실감나요.다만 크기가작으니 분실위험이 있어 불안해요.욕실도 가지고 들어가면 안될거같구...길가면서 음량을 너무 크게 켜면 자동차소리가 잘안들리니 주의해야할것같아요. 5 

5 사실 다른 코드프리 이어폰들과 비교해서 비싼 가격이라고는 생각되지 않습니다.
다른 제품들을 많이 사용했지만 에어팟만큼 완성도가 높고 편리한 제품은 많지 않다고 생각합니다.
다른 애플기기들과 사용할 때에는 그 어떤 이어폰들이 상대가 되지 않을 정도로 편리합니다.
음질도 블루투스로서는 만족할만한 수준이고, 배터리도 오래가고, 충전할수 있는 케이스에 탈착하는 것도 다른 이어폰들보다 정말 편하고 깔끔합니다. 5 

6 왜 무선이 답인가!!! 에어팟 첫 이미지는 너무 작고 분실의 위험이 떠오르는데요..막상 사용시 귀에 쏘옥 들어가 안정감 있고, 비사용시 케이스에 넣고 다니니 분실의 위험이 없더라고요. 음질은 기본이구요...밧데리도 케이스에보관하면 충전되는 시스템이라 여유있게 사용이 가능한것 같습니다. 4 

7 사용하다가 넘 편해서 동생 선물하려고 샀습니다~
품절이라 계속 기다렸는데 재입고가 되었더라구요~
바로 구매하고보니 금방 또 품절되었던데
빨리 구매하길 잘한 것 같아요ㅎㅎ
롯데멤버십행사까지하니 좋아요
하이마트는 믿음직하니까 타사이트에서 사는 것보다 맘편히 살 수 있었어요 감사합니다~ 5 

8 처음에 출시되었을때는
이해 못 하는 비싼 콩나물이라고만 생각했었는데 막상 구매해서 사용하니 너무 편리하고 좋아요!
토요일 10시 넘어서 주문했는데 일요일 오후에 배송 출발! ㅜㅜ 주말인데 주문 내역확인하고 빠른 배송 해주셔서 오늘 아침에 받았어요 너무 감사드려요~ 감기 조심하세요✋🏻 5 

9 배송도 엄청 빠르고 오자마자 착용해봤는데 딱이었어요ㅎㅎ 집에 고양이가 있어서 매번 이어폰 끊어지고 이번이 네개째였는데^^...이건 관심도 안보이

4 갑자기 에어팟에 꽂혀가지고 여기저기 알아보다가 하이마트에서 파는걸 발견했는데 청구할인이랑 l포인트도 적립되고해서 나쁘지않은 금액으로 잘 산 것 같아요!ㅎㅎㅎ
배송도 빨라서 좋았구요 꼼꼼하게 잘 포장해서 주셔서 감사해요~! 에어팟은 성능도 좋고 잘 작동하구요 그냥 보기만해도 너무 귀엽고 예쁘네요ㅎㅎㅎ계속 끼고있고 싶어요~~ 착용샷도 괜찮고! 진짜 비싸서 고민 많이했는데 그만큼 만족감도 커서ㅋㅋㅋㅋ주변사람들한테도 추천중이에요!!ㅋㅋㅋ 5 

5 디자인이야 두말 할것 없이 깔끔 그 자체예요. 얼핏보면 귀걸이 한것 처럼 보이기도 ㅋㅋ 기기상에는 작동 버튼이 없어 통화하거나 볼륨을 조절하거나 할때는 핸드폰으로 해야 하는 불편은 있지만 반응이 엄청 빠르고 통화 음질 아주 좋고 디자인이 예뻐 불편함은 패스 ㅋㅋ 무선이고 고정장치없이 귀에 꽂는 스타일이라 사람들하고 아주 쎄게 부딪히거나 하면 귀에서 이탈되어 조심할 필요성은 있어요.... 5 

6 배송도 빠르고 물건도 짱짱합니다~!
에어팟 생긴 건 약간 콩나물 대가리 같긴 한데 그래도 이만하면 삶의 질을 올리기에는 충분해요!
애플 특유의 깔쌈한 디자인과 군더더기 없는 심플함이 물건을 더 돋보이게 합니다! 운동이나 산책을 즐겨 하시는 분에게 강추! 근무시간에 몰래 음악듣기 개꿀이염~ 5 

7 음질도 좋고 디자인도 이쁘고 ㅋㅋ 저 같은 애플빠들에게는 안 사기가 힘들 정도네요ㅠㅠ
비싼 가격에 망설였지만 돈이 하나도 안깝습니다!
여러분 절대 귀에서 안빠져요! 헤드뱅잉 해도 안떨어집니다.
비록 귀에 꽂은 모습이 폼은 안나지만, 엄청 안정적이고 음질도 좋고 대만족입니다! 완전 강추합니다!! 5 

8 역시 애플입니다.
디자인도 깔끔하고, 무엇보다도 아이폰과의 페어링은 정말 그야말로 눈깜박할새 이뤄집니다.
애플의 최강점인 기기간 연동은 정말 타사가 모방할수 없는 순준입니다.
막귀라 그런지 몰라도 음질도 칭찬할만 하구요. 구매하길 잘했다는 생각이 듭니다.
밍설이시는 분들께 돈이 아깝지 않다는걸 말씀드리고 싶네요.
애플 유저라면 놓치

130 이쁘고 깔끔하고 최고네요 5 

131 에어포드 비싸지만 잘 구입했어요
만족 스러워요.

좋아요.
에어포드 무선 충전 되는 제품이 나온다고 하긴 하지만 지금으로서는 최선이니 5 

132 디자인은 별로데 성능은 생각보다 괜찮음 3 

133 매우 편의성이 좋고 음질도 수준급입니다. 디자인도 좋고. 편의성도 매우 좋습니다. 무선이라 걱정했는데 음질도 수준급입니다. 매우 추천합니다. 5 

134 품질이야 뭐... 워낙 우수하니깐.. 두말 필요없고 정말 디자인 좋고 편리함. 단순한 사용 방법 그리고 쉬리 활용도가 높음!!! 11번가 덕에 저렴한 가격에 빠른배송에 굳굳 ~~ 5 

135 고민하다가 구입했는데 생각보다 크기가작아서 놀랐구요 디자인이 깔끔해서 좋았고사용해보니까 음질도 괜찮고 착용한느낌이 거의없을정도로 좋네요 5 

136 아주 편리하고 베터리충전이 가능해서 오래쓸 수 있습니다
디자인 호불호가 갈릴 수는 있지만 저는 좋더군요 5 

137 배송빨라요 아주빨라요 일단 품질도좋고 생각보다 훨신 만족스럽네요 에어팟 강추 모양새가 그닥 이쁘진않디만 정말 편하고 좋네요 굳 5 

138 디자인도 심플하고 음질도 떨어지지 않고 휴대성도 좋아서 아주만족해요 가격빼구요...^^ 안산사람은 욕해도 쓰는사람은 만족한다더니 만족스럽네요 ㅋㅋㅋㅋ 5 

139 완전 내스타일이고, 디자인마음에들고 편안해서좋아요
단점은 비싸다는것과, 롯대배송이 느리다는거 빼곤 ㅜㅜㅠ 4 

140 이건 혁신입니다!!!!역시 사람들이 선택하는 이유가 있었어요- 음질 완전 깨끗해요- 모양도 이쁘고요- 역시 애플에서 벗어날수가 없어요ㅋㅋ 5 

페이지 8 

141 아직 써보지는 않았지만 일단 모양부터 마음에 드네요~~~ 5 

142 디자인도 깔끔하고 아무래도 애플 계열이다 보니 일반 블루투스 이어폰 보다 좋은 것 같습니다 ㅎㅎ 강추 ㅎ 5 

143 배송도 빠르고 역시 깔끔하고 좋아요..타 블루투스 이어폰보다 당연히 호환도 좋구요 5 

144 ㅠㅡㅜ 완전 대박... 진짜 운동하는데 완전

## 에어팟 2세대

### 에어팟 1세대와 2세대의 가장 큰 차이는 W1->H1칩 발전

H1은 동기화가 개선됐다. 즉, 에어팟으로 전화를 받을 때 훨씬 빨리 연결되고, 다른 디바이스로의 연결도 더 빨리 진행된다. 애플에 따르면, 에어팟은 디바이스간 연결 전환이 최대 2배 빨라졌고, 통화 연결 시간이 최대 1.5배 빨라졌다고 한다.(품질,성능)

통화 가능 시간도 더 길어졌다. H1의 배터리 효율이 높아졌기 때문이다. 1세대 에어팟은 통화 가능 시간이 2시간이었는데 H1 칩을 탑재한 2세대 에어팟은 총 3시간이다. 음악 감상에는 최대 5시간을 사용할 수 있으며, 케이스를 활용해 총 24시간을 사용할 수 있다. 15분 충전으로 3시간의 음악 감상 및 2시간의 통화가 가능한 수준으로 충전할 수 있다.(배터리 수명)

H1은 현재 귀에 착용하고 있는 에어팟의 수(한쪽인지, 두쪽인지, 아예 없는지)를 광학 센서와 모션 가속도계에서 받은 정보로 파악한다. 그 다음 여기에 맞춰서 오디오를 전송하고 통화할 때 외부 노이즈 감소 시스템을 조정한다.(음질,기능,품질,성능)

더불어 1세대 에어팟의 문제점, 특히 게임을 할 때 문제점 중 하나로 꼽혔던 레이턴시(latency=반응속도)가 감소했다. 애플에 따르면 H1 칩으로 레이턴시가 최대 30% 감소했다.(성능)

음성으로 시리를 호출할 수 있게 된 것도 큰 개선점이다. 이제 아이폰을 들지 않아도 “시리야”라고 말함으로써 음악 재생을 제어하거나, 전화를 걸거나, 메시지를 보내거나, 방향을 알 수 있다.(조작성, 사용성,기능)

음질, 디자인,착용감,휴대성 차이 없음.

만족도(카테고리 있음)비교도 좋을듯

원문보기:
http://www.itworld.co.kr/news/119204#csidxd40396c08da7bd6bcac69035a026bc9 

In [10]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','사용성','조작성','디자인','착용감','휴대성']

#에어팟 2세대 검색 (무선)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nv_mid=18754856889&cat_id=50002334&frm=NVSCMOD&bt=0&query=%EC%95%A0%ED%94%8C+%EC%97%90%EC%96%B4%ED%8C%9F+2%EC%84%B8%EB%8C%80&NaPm=ct%3Dkbmh9o0w%7Cci%3D80368a4047f35fbf8923255886cb45e796cf75d0%7Ctr%3Dsls%7Csn%3D95694%7Chk%3Db51adcb37057ddfb9e00a0bc6040b672cd39731d"

shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[9]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[15]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[12]/a/span"  #사용성
category7="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[13]/a/span"  #조작성
#향상되지않은기능
category8="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[5]/a/span" #디자인
category9="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[10]/a/span" #착용감
category10="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[14]/a/span" #휴대성



header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/jmlee/Downloads/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df2=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df2.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df2.loc[n]=[name,category,'null','null']
        n+=1    
    return df2

def save():
    if not os.path.exists('output3.csv'):
        df2.to_csv('output3.csv', mode='w')
    else:
        df2.to_csv('output3.csv',mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"성능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"기능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"배터리 수명" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"사용성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[6]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
# d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

페이지 1 

1 배송도 빠르고 정품인것 같네요^^
근데 정품등록이 안되서;; 안해도 별상관은 없다지만 그게 좀 아쉽네요^^
제품자체는 맘에듭니다. 페어링도 잘되구요. 근데 마감이 좀 안좋아서 귀안쪽으로 깊이 넣으면 긁혀서 아픕니다ㅠㅠ.
그래도 두손이 자유로워져서 너무 편하고 통화품질도 좋은것 같아서 만족합니다! 5 

2 음질이 생가보다 좋았어요. 특히 베이스가 빵빵한것이 맘에 듭니다. 무선이라 편하고요. 갤럭시라 따른 앱을 받아서 설정을 했는데 배터리가 좀 빨리 다는것같았어요. ff기능은 터치말고 버튼이나 모 이런걸로 했으면 더 좋았을거같은데 그래도 쓸만하더라구요. 5 

3 카드사 행사때 저렴하게 구매해서 가격이 만족스럽습니다 여러 키링과 에어팟케이스로 꾸며서 예쁜 백참처럼 이용하는 재미도 좋습니다. 무선이어폰은 정말 신세계네요 각인서비스를 이용하지 못하는 단점이 있기는 한데 뭐 보통 에어팟케이스를 쓰니 구지 각인서비스가 필요는 없겠다 싶어서 그냥 구매했어요 단지 현재 이용하는 빈도에 비해 배터리가 빨리 닳아서 애플센터에 한번 방문해보려구요 뽑기가 잘못된건지 좀 확인해 봐야겠네요 가격은 좀 비싸지만 가급적 아이폰은 제 짝꿍인 에어팟을 이용하는게 개인적인 느낌에서 좋은것 같습니다 음질도 너무 좋아요 그리고 어느정도 거리까지는 멀리서도 이어폰 이용이 되니 너무 편리합니다. 카드사 행사나 쿠폰등 이용하면 좋은 가격에 살수 있으니 추천합니다. 유선충전과 무선충전 다소 고민했는데 무선충전이 확실히 편리함이 있습니다. 몇마넌 차이가 나지 않기때문에 . 약간 귀에서 떨어져서 잃어버릴수도 있겠다 싶긴 한데 요즘 예쁜 색상 이어캡이 많아서 꾸미기도 할겸 실용성 있게 구매해 이용해 보려고 합니다. 5 

4 유무선 중 고민하다 무선으로 구매했어요. 뭐 음질이 너무 좋다해서 기대 많이 했는데, 사용하고 있는 유선 이어폰하고 비교해도 월등하다고 할 정도는 아니네요. 가격따지면, 기대만큼 지불할 정도 아닙니다. 무선쓰다 유선 쓰니 불편해서 사긴 했는데요. 사실 분들, 에어팟 구

116 빠른배송에 한번 감동, 제품 성능에 또 감동 했어요생각보다 귀에 답답한 느낌없이 잘들리고 좋아요 음질도 굿 5 

117 음질도 좋고 편하고 완전 만족합니다.배송도 무지 빨랐구요^^ 5 

118 자주 애용할게요
음질도 짱좋아요 잘샀네요 5 

119 그동안 고민했던 게 시간이 아까울 정도네요
무선이어폰은 정말 신세계네요
음질도 좋고
가격은 하이마트가 젤 쌌어요!!
배송도 빨라서 아주 만족합니다 5 

120 배송도 빠르고 음질도 좋아요 5 

페이지 7 

121 음질좋고 짱 입니다 .. 5 

122 갑자기 한쪽이 안들려서 잠시 당황했지만 인터넷서치해서 고쳤네요. 이런 문제만 안 생기면 음질도 좋고 다 괜찮아요. 남편꺼 하나더 사러 들어왔네요 4 

123 중2 아들이 용돈모아서 산 에어팟입니다!
비싸서 다른제품과 고민 많이 해서 산것같은데ᆢ
역쉬~~ 굿초이스🤗
오자마자 엄청 좋아하네요!
음질도ᆢ 굿! 간지납니다~ 5 

124 근데터치잘안되고구린것가타여
진심불량인가싶더라구요
음질은좋고사용감이줄없어서편함 3 

125 역시 애플입니다! 음질 및 편의성이 넘 좋아요~ ㅅ ㅏㅇ품 잘받았습니다~ 5 

126 별 설정 없이 아이폰에 바로 인식되네요.
음질도 훌륭합니다.
모처럼 아빠노릇 한번 제대로 했네요. ^^ 5 

127 무선충전도 잘되고
음질 좋아요
통화도 제일 나은거 같음 5 

128 배송 정말 빠르네요.음질 정말 좋습니다. 5 

129 배송도 빠르고 음질도 너무 좋아요~ 5 

130 선
넘 맘에 들어요...ㅋㅋㅋㅋㅋ
선 없는 게 이리 편할 줄이야ㅜㅜ
무선충전도 잘되네염ㅎㅎㅎ
확실히 2가 음질이 더 좋네용ㅎㅎㅎ
앞에 불빛이 표시되는 것도 좋아요
5월제조된 거라서 기분 훨 좋아요ㅋㅋㅋㅋㅋ 5 

131 빨리 와서 좋고 음질도 좋고 !! 5 

132 살까말까 고민하신다면 사세요 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ처음엔 에어팟보고 저 콩나물 뭐야 이랫는데.. 너무편하고 짱좋아용
👍👍👍 음질도 최고. ㅠㅜㅠ 5 

133 음 제가 샀을때보다 가격이 더 떨어

8 기능 다 잘되고 좋았어요- 내일 정품 인증 해보려고 합니다 :) 5 

9 기능 맘에 들어요 4 

10 예상했었지만 기능과 디자인, 색상 모두 만족합니다. 착용감도 좋고 AI 기능도 탁월합니다. 5 

11 주문한지 하루만에 배송되서, 기능과 디자인에 만족하며 잘 사용하고 있습니다. ^^ 5 

12 엄청 초고속으로 받았어요^^귀도 편하고 너무좋아요
성능기능최고 5 

13 와우*^^*~ 매우 만족해요.
편리하고 기능 짱이네요~ 5 

14 목욜에 온다 했는데 화욜에 왔어요!
갤럭시라서 제한되는 기능 있어서 슬프지만 제품 자체는 너무 좋아요!!!! 고민 많이 했는데 사길 잘한거 같아요~^^ 5 

15 무선충전 기능이 너무 좋네요 5 

16 역시 너무좋아요 음질좋고 빠르고 기능많고! 5 

17 품절풀렸다는 문자 받고 바로 주문했더니 다음날 배송해주셨어요~ 간편하고 페어링잘되서 넘 좋아요! 5 

페이지 1 

1 편하고 배터리 오래가서 좋네요 4 

2 잘되네요
통화도 괜찮고 터치도 잘되고
너무 편해요
배터리도 생각보다 오래가고
고장만 안나면 좋을것같음.ㅎㅎ 5 

3 끊김없습니다.
무선충전 좋습니다.
배터리 오래갑니다
디자인 똑같습니다.
음질 똑같습니다. 5 

페이지 1 

1 정말 좋아요. 진짜 신세계네요.
블루투스 이어폰이랑 헤드셋이 4개나 있는데, 그 동안 느껴보지 못한 편함과 성능이네요.
다른 이어폰이랑 헤드셋은 활동하면서 자주 빠지기도 했는데, 더 잘 빠질거 같이 생겼는데 안 빠지네요. 그래서 헬스장 가서 사용하기도 더 편해요.
블루투스 연결할 때는 왜 이리 간편한지, 아마 한번 사용해 보고 운이 없어 잃어버리기라도 하면? 다른 아이템으로는 못 옮길 듯 합니다.
계속 사용하는 게 아니라 한 1주일에 한번씩 충전할라나... 가끔 사무실 무선 충전기 위에 올려 놓고 나중에 사용하면 다시 1주일은 사용하는 듯 하고요.
다만 에어팟 자체에 볼륨 조절은 안 되어(내가 못 찾았나요?) 그것만 좀 불편하네요.
귀에 쏙 들어가는 간편함. 타사 제품처럼 

4 에어팟2 저렴하게 구매할수 있었어요.
애플 이어폰은 처음 구매해보는데
착용감이 좋고 통화할때 좋네요. 5 

5 잘 받았습니다
착용감이 역시 기존 이어폰과 비슷해서 좋네요 5 

6 저렴하게 구입 빠른 배송 마음에 듭니다 ㅎ
버즈보다 착용감이 좋네요 5 

7 뱅앤울룹슨 E8을 사용했었습니다 E8과 비교하여 착용감이 월등히 우수하고 블루투스 연결 안정도, 사용 편의성 또한 났다고 생각됩니다. 5 

8 예상했었지만 기능과 디자인, 색상 모두 만족합니다. 착용감도 좋고 AI 기능도 탁월합니다. 5 

9 착용감도 좋고 정품이라 더 좋아요 5 

10 아이폰을 쓰고있어서 기존에는 엘지꺼 블루투스 목에걸어섰는데 고장나서 에어팟2세대 나왔길래주문했어요 배송은 어제 올줄 알았는데 오늘 받았어요성능은 좋은데 귀가 작아서그런지 귀가 아파요 3 

11 배송 엄청 빨리왔네요 무선이라 엄청 편하네요 착용감도 좋구요 5 

12 착용감 음질 모든게 너무 다 좋아요 5 

13 카드사할인까지 받아서 싸게 샀어요 소리가 너무 좋고 귀도 편하고 착용감이 좋아 분실 우려도 없겠네요 5 

14 생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다 5 

15 우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어팟이 더 착용감이 좋네요
기어 아이콘 쓸때는 귀가 아파서 장시간 사용이 어려웠는데 에어팟은 통증이 없어서 끼고 자도 아프지가 않네요 5 

페이지 1 

1 휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 구매했습니다. 평소 아이폰, 아이패드 미니, 아이패드 프로를 휴대하며 상황에 맞게 사용합니다. 하지만 기기를 옮겨 무선 헤드폰을 연결할 때, 기기를 끄고 페어링 모드로 변경해서 다시 연결하는 절차가 너무 불편하더군요. 그래서 기기 사이의 페어링이 간단한 제품이 필요했습니다.
에어팟은 다른 애플 기기에 연결되어있는 상태에서도 연결하려는 기기의 블루투스 설정에 들어가 에어팟을 누르기만 하면 쉽게 페어링이 된다는

In [1]:
#에어팟 2세대
#패키지 import

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

name=['에어팟 1세대','에어팟 2세대','에어팟 2세대(유선)']
category=['음질','품질','성능','기능','배터리 수명','사용성','조작성','디자인','착용감','휴대성']
#에어팟 2세대 검색 (유선)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=18622086330&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25202%25EC%2584%25B8%25EB%258C%2580%2520%25EC%259C%25A0%25EC%2584%25A0%25EC%25B6%25A9%25EC%25A0%2584&NaPm=ct%3Dkbb1kefs%7Cci%3D145113ad84d3bb4c745b6faa37d4d9894a9d33fb%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dad9562496841c80a6f639b5837f07d5e88b01927"

shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[3]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[17]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[10]/a/span"  #사용성
category7="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[15]/a/span"  #조작성
#향상되지않은기능
category8="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[5]/a/span" #디자인
category9="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[11]/a/span" #착용감
category10="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span" #휴대성



header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/jmlee/Downloads/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df2=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df2.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df2.loc[n]=[name,category,'null','null']
        n+=1    
    return df2

def save():
    if not os.path.exists('output4.csv'):
        df2.to_csv('output4.csv', mode='w')
    else:
        df2.to_csv('output4.csv',mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[2]  #에어팟 2세대
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"성능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"기능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[2]  #에어팟 2세대
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"배터리 수명" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"사용성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[6]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"디자인" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[2]  #에어팟 2세대
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
# d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[2]   #에어팟 2세대
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


페이지 1 

1 안드로이드 폰만 사용중이예요. 그래서 구매를 망설였는데 연결 잘 되고 소리도 잘 나네요. 이어팟 음질에 실망해 의심스러웠던 음질은 나름 괜찮습니다. 사용 편의성은 다른 코드리스 이어폰을 사용해 봐서 그런지 뺐다 꼈다 하기가 좀 불편하네요. 기본 구성이구 최저가격은 아니지만 한 번에 두 개 살 수 있는 곳이라서 구매했어요. 배송 빠르고 2019년 9월 생산품으로 잘 받았습니다. 비싼 감이 있어 망설였지만 사람들이 왜 이것을 사게 되는지에 대한 인지는 바로 하게 되었구요. 편리하고 예쁩니다. 5 

2 @가격 : 에어팟 2 요즘에 그냥 다 이 가격대 인것 같아 싸게 산것 같진 않고 그냥 보통? 그냥 그 가격; 이미 프로도 나왔으니...
@기기 : 뭐 요즘 짭 많다 하는데 내가 짭을 경험 해본 경우가 없어서... 근데 수령하고 에어팟 열면 아이폰 같은 경우 자동 등록 되는것 같고 뭐 상세한 설정 같은것; 아이폰 사이트 보증 등록, 본인 기기 설정, 등 만 조금만 하면 되고 기기 자체는 상태 괜찮은 듯 하다. 나는 조금 늦은 3월에 샀는데 20년 01월 제품으로 들어옴.
@에어팟 프로와 비교 : 물론 개인 만족 이지만 나는 실제 프로와 2 둘다 봤는데 솔직히 프로가 더 작고 디자인도 더 편해보이고 작아서 디자인 면에서는 프로가 괜찮지만 나는 개인적으로 커널형 이라지만 앞에 고무 달린 이어폰을 개인적으로 별로 안좋아 해서 ; 이물질 같은것도 더 많이 묻고 닦다 보면은 고무도 늘어나고 에어팟 기존의 형태가 . ...
@무선과 유선 : 요즘에 뭐 무선 충전 많이 하고 카페 같은데 가도 무선 충전기 들이 다 배치 되어 있어서 편하긴 하지만. 솔직히 프로 사는거 아니고 나와 같이 에어팟2 사는데 무선 유선 고민 한다면 뭐 갠취 지만 나는 유선 추천. 어차피 이 허얗고 조그만 한거 그 카페에서 무선 충전 한다고 냅둬 봤자 언젠가 잃어 버린다. 어차피 집에서 충전을 가장 많이 할텐데 둘중에 사야 한다면 그냥 유선 사는게 그나마 가성비다. 5 

3 감사합니다 판

18 좋은 가격에 잘 구매했습니다. 두번째 구매입니다.케이스주는 옵션있는 상품이 랜덤이라 불필요하고 맘에들지않아서 이곳에서 구입했는데 만족합니다. 명성대로 음질좋습니다.가격이 비싼만큼 오래도록 as안받고 잘 쓸수있기를바랍니다 3 

19 사용해보니 음질은 좋더라구요 통화시 음질도 좋고 음악들을때 음질도 좋구요 사용성도 편하더군요
생일선물했는데 만족한답니다 생각보다 작구요 디자인은 평범합니다 너무 작아서 분실의우려거 잇으니 잘 보관하셔야 하겠더라구요 케이스도 작아서 쉽게 잃어 버릴수 있겠어요
당일배송이 당일 받을수 잇다는 말은 아닌가 봐요 제가 이해를 잘못했나봐요 당일배송이라고 해서 당일받을줄 알았는데 한 3일 정도 배송기간이 소요되더라구요
어쨋든 선물받은사람이 만족한다니 다행입니다 에어팟의 모든기능을 충분히 사용할려면 아이폰과 연동해서 사용하는게 더 좋다네요 엘지 핸드폰이라 지원한되는 기능들이 잇나봐요 이점 참고하세요 3 

20 애들 에어팟 1세대 3년정도 주구장창 쓰니 결국은 센써가 고장나더군요~
귀에 꽂혀 있는걸 인식 하는 센서가 고장이라는 서비스센터 직원 분의 말씀~! 새로 사는게 낫답니다~!
결국 2세대를 다시 구입~!
저는 갤노트 시리즈만 사용하는 일인이라 편리성이 조금 떨어지지만 그래도 무선중에서는 편리성 기능 음질 최고인듯~!
커널형이 맞지 않는 제 귀에는 에어팟이 딱!
다만 길을 걸을때 한쪽귀만 꽂은채로 사용하니 두개의 동시 페이링이 불균형해 지는듯!
그래도 좋음~ㅋ 5 

페이지 2 

21 귀에 넣으면 띠링 소리와 함께 음악이 흘러나온다.
다른것보다 통화품질은 대만족인데 전화기를 들고
전화할때와 크게 차이가 없다.
손목 터널증후군때문인지 쑤실때도 있었는데
에어팟 덕에 병원비 아꼈으니 속된말로 핵이득이다.

거기다 수신범위도 여타 블투이어폰과는 다르다.

이전에 써본 제이버드와 비교도 안된다.
사무실에 폰을 두고 5미터정도 떨어진 화장실을
가도 수신이 안끊긴다.

아침 해피타임에 휴대폰과 잠시 떨어져
노래를 들으며 집중하면

치질도 이제 걱정

54 에어팟 케이스가 너무 갖고 싶어서 고민끝에 에어팟 산 안드로이드 사용자입니다!제 폰은 V30인데 맨 처음 페어링도 잘 됐구요 어플 깔았더니 배터리확인도 잘됩니다.이제까지 커널형을 오래 써왔는데 커널형 못지않게 소리선명하고 너무 좋아요! 왼쪽 두번탭해서 노래 멈추는것도 되더라구요.정품등록 완료했습니다.비싼만큼 오래쓰고 싶어서 철가루스티커랑 케이스도 장만했어요.그런데 끼고있으면 왼쪽귀가 아프네요.. 5 

55 첨부사진 설정샷 같이 생기긴 했는데 제가 찍은거구여. 음질 진짜 좋아요! 처음 착용하고서 기묘한 이야기 보는데 드라마에서 나오는 소리를 밖에서 들리는 건 줄 착각하기도 하고, 노래 들을 때도 훨씬 좋아요 기존 아이폰 이어폰(?) 보다는요! 그리고 시리 부르는 것도 편하고 에어팟 잠깐 빼면 노래나 영상 정지되는 것도 좋네여. 전화할때도 편하고. 편함 그 자체에요! 사은품 검정색 바랐는데 하늘색 케이스 온 게 조금 아쉽지만요ㅎㅎ 5 

56 너무 좋아요! 저는 매장에서 살까 조금 고민하다 당일배송이라는 말 믿고 가격이 좋아서 여기서 주문했어요!
연말이고 연휴가 껴있어서 배송이 조금 늦었어요
그래도 빠른배송 신경 써주셔서 감사해요
빠른배송이고 매장가 보다 훨씬 착한가격으로 선물 해주니 받는 사람도 좋아하네요
음질도 좋아요! 불량이 올까봐 조금은 걱정은 했는데
문제 없는 제품이여서 다행이네요! 그리고 정품등록도 했어요
매장에서 안사도 되겠어요 이제
이리편할줄이야 삶의질이 향상된다하니 좋으네요
고민하시는분들은 고민하지말고 빠른결정 선택하세요
간편하고 운동할때도 걸리는게 없으니 좋아요
저는 매장에서 더 비싸게 주고 ?지만
좋은가격으로 살 수 있어 좋네요
많이 파세여너무 좋아요! 저는 매장에서 살까 조금 고민하다 당일배송이라는 말 믿고 여기서 주문했어요!
연말이고 연휴가 껴있어서 배송이 조금 늦었어요
그래도 빠른배송 신경 써주셔서 감사해요
빠른배송이고 매장가 보다 훨씬 착한가격으로 선물 해주니 받는 사람도 좋아하네요
! 불량이 올까봐 조금은 걱정은 했는데
문제 없

95 잘 받았어요!

급하게 필요해서 주문 했는데 바로 받을 수 있어서 좋았습니다.

뾱뾱이 포장도 해주시고 안전하게 왔어요! 2세대 정품이고 애플 홈페이지에서인증도 잘 완료 했습니다~~~~~~~
큰돈 들여서 구매한 보람이 있습니다.
만족스러워서 별 다섯개 드립니당
많이 파세요 5 

96 마감이 들떠있다는 후기가 많아서 혹시 가품인가하고 걱정많이했어요 역시나 받아보니 마감이 좋지는 않더라구요검색해보니 중국으로 생산라인 바뀐뒤로는 거의 이렇다더라구요 매장가서 산 친구것도 보니까 마감이 똑같길래 가품걱정은 없어졌어요ㅋㅋㅋ음질도 좋고 배송도 빨랐어요~ 4 

97 배송은 하루만에 도착해서 너무 좋았어요~ 편리한건 두 말할 것 없이 최곤데 생각보다 음질이 그닥.. 좋진 않네요 다른 에어팟을 사용해본 적이 없어서 단정짓긴 어렵지만 전에 쓰던 정품 애플 이어폰에 비해서는 음질이 떨어져요 제가 받은 제품만 이런건지 원래 이런건지..ㅠ 3 

98 금요일늦게 주문해서 월요일배송예정이라고해서 기다리고있었는데 바로 토요일날 배송이되었어요..딸아이 깜짝선물이었는데..너무너무 주말동안행복해했구요..한쪽씩끼고 산책도나가고.좋았어요..음질도깨끗하고 잠깐씩 끊기는거조금있었는데.문제될정도는아니구요.상품도 6월제조일자거로왔어요..삼성폰과도 연동잘되구요.비싼만큼 고민도많이하고 그랬는데.좋아요..딸아이보다제가더좋아요. 5 

99 운동할때사용하던 3만원대사용하다가 저렴하게 행사해서 구입했어요. 음질이 짱이네요. 귀에 꽉끼지도 않고 귀가 편하다고 해야할듯. 좋네요.저렴한거는 상대방이 통화음이 나쁘다고들 했는데. 이건 음질이 좀다고하네요. 비싼만큼 이유가 있는듯 하네요. 강추합니다. 5 

100 처음 도착했을 때 바로 확인은 못했고 나중에 확인했는데 왜 사람들이 에어팟을 사용하는지 알게되었어요. 진짜 음질이 선명해요! 아이폰이랑 아이패드에는 엄청 선명한데 삼성폰에는 연결안해봐서 아직 잘 모르겠어요! 그래도 엄청 만족합니다!! 대신에 유선은 2세대이지만 1세대랑 차이가 안나서 맨처음에는 놀랐어요

177 음질이 좋아요. 잘 쓸게요. 5 

178 통화품질 아주 좋습니다 5 

179 가격은 좀 비싸지만 음질도 좋고 귀에 착용감도 좋고마감도 깔끔하고 만족스럽습니다 4 

180 처음 페어링 때는 찢어지는듯한 소리가 났었는데, 이후 괜찮았기에 상태를 지켜보고 있습니다. 제조년도 20년이고, 포장도 꼼꼼히 해주셔서 만족합니다. 5 

페이지 10 

181 에어팟 쓰니까 삶의 질이 달라지는 것 같아요ㅎㅎ 너무 편하고 좋네요 음질도 너무 좋아요 잘샀어요~~ 배송도 예정보다 빨리 왔네요 만족합니다! 5 

182 없을땐 몰랐는데 있으니깐 너무 좋네요!아이폰에 등록도 잘되고 음질도 좋아요^^배송도 뽁뽁이에 쌓여서 잘 왔어요! 4 

183 역시 음질최고입니다.
비싼만큼 그 값어치를 해주네요.
잘 쓰다가 업그레이드 버전 재구매 해야겠어요 5 

184 비싸지만 선이 엉킬일도 없고 소리도 깨끗하게 잘 들리는 것 같아요 ㅎㅎ 5 

185 아주 저렴하게 잘 구매했습니다음질도 좋고 만족스러워요 5 

186 음질도 좋고 연결자체도 문제가 없이 깔끔해서 좋았어요~ 5 

187 음질좋고 쿠폰이랑 상품권으로 저렴하게 구매했어요 5 

188 넘 잘 들리구 음질도 좋아요! 1세대는 어땠는지 몰라서 비교는 어렵지만 전 2세대 완전 만족! 5 

189 갤럭시노트9에 블루투스 연결 잘되고 음질 좋아요. 특히 통화음질도 우수. 강추합니다. 5 

190 아 정말 신세계네요~~~ 음질도 좋구 편해요! 5 

191 역시 음질 짱 고장없이 잘사용 하고있네요 5 

192 소리도 잡음없이 잘들리고 좋아요 5 

193 음질도좋고 디자인도 너무 좋아요 5 

194 확실히 편하고 소리도 좋네요 4 

195 음질도 좋구 너무 좋아요. 5 

196 정품 등록햇고 음질 좋고 다 괜찮네요~ 맘에들어요~ 케이스도 같이 구입해서 씌워?어요~ ㅎㅎㅎ 5 

197 구매일자등록이 안되서 고생하다 고객센터에 전화햇더니 등록 잘되어잇다시더라구요 정품등록해서 보내신건지 암튼 음질좋고 선없는 편리함이 이런

...

3 기본 10자 이상 작성시 100P 즉시지급 이미지 등록시 100P 즉시 추가지급개당 상품 가격이 5,000원 이하 상품은 적립대상에서 제외되며,리뷰 포인트는 최초 1회 작성시에만 지급됩니다.같은 단어가 반복된 내용은 임의로 삭제되며, 지급된 포인트가 회수될 수 있습니다. 다른 고객님들에게 유용한 상품 리뷰를 작성해주세요.배터리도 오래가고 줄없으니까 너무 편하네요~~리뷰 이제 생각나서 씁니다 5 

4 일단 선이 없다는게 제일좋네요. 어차피 선 없는 이어폰이 필요했구 에어팟외형이 맘에 들어 구입했으니 후회는 없습니다~음질은 유선이어폰과 흡사해요. 통화시 주변소음 상대에게 들린다고하네요. 모바일게임 중 가끔 블투연결이 끊키곤 해서 주변에 겜소리 들리게하여 민망하네요.배터리가 생각만큼 오래가지 않아 자주충전 해주고 있어요. 5 

5 유선,무선중에 고민고민하다 결국 유선을 선택했는데 잘한거같아요.
충전도 빨리되고 배터리도 오래가구요.
커널형이 안맞는 귀인데 오픈형이라 귀도 편하고 좋습니다. 5 

6 너무 편해요! 귀에 끼면 자동 연결, 음악듣다 한쪽을 빼면 자동 정지. 음악듣던중 전화연결. 배터리도 오래가요. 다 만족스러워요. 5 

7 에어팟1 사용하다가 배터리가 2시간도 안가서 이번에 재구매 했는데바뀐건 없지만 배터리가 오래가서 좋아요. 많이파세요. 5 

8 잘 들리네요 1보다 배터리도 오래가구 5 

9 역시 에어팟 음질이 최고입니다. 그리고 놀라운건 배터리 진짜 오래갑니다. ㅋㅋㅋ 적극추천 5 

10 특히
무선 이어폰 없어서 구매했는데, 음질도 괜찮구요
특히 자동 페어링이 편하네요. 배터리도 오래갑니다 5 

11 유선이어폰은 사면 언젠가는 한쪽이 끊겨서 개빡쳤는데 이건 진짜 좋은듯. 가격도 싸고 배터리도 정말 오래감. 5 

페이지 1 

1 주문하고 다음날 바로 배송받았어요~~ 쿠폰할인에 카드청구할인까지 받아서 타사이트보다 굉장히 저렴한 가격에 구매했어요~!! 받자마자 정품등록도 완료했어요 페어링도 에어팟1보다 빠르고 베터리도 음악만들으면 굉장히

특히 자동 페어링이 편하네요. 배터리도 오래갑니다 5 

84 주문한 다음날 바로 받았어요
제품은.. 19만원 값어치 못하는 품질이었어요.
유격이 있고 제품 마감이 좋지 않네요.
이거야 뭐 하이마트 문제가 아닌 애플 문제니깐..
무튼 사용은 편합니다 3 

85 잘받았어요무선이어폰 편하네요빠른 배송 감사합니다잘황용해볼게요 5 

86 음질 사용성 정말 최상입니다. 갤럭시폰으로도 사용하기 편합니다 5 

87 주문하고 다음날 바로 받았어요~!
정말 사용하기 편한 것 같아요 ㅎㅎ
가격도 하이마트가 이것저것 할인받으면 싼 것 같아요. 너무 만족해요~! 5 

88 비싸서 문제이지 사용도 편하고 하고 음질도 좋아요. 배송 엄청 빨라요 4 

89 단점꼽으라면 이어폰자체에 볼륨조절없는거 빼곤
기능도 좋고 음질도 좋습니다ㅎㅎ
바로 블루투스 연결되서 편리해요 5 

90 너무 편하고 시리랑 같이 사용하니 편하네요
진즉 살 걸 그랬다는 말이 이해되네요
하이마트 배송도 빨라요 5 

91 예전부터 벼르고 있다가 결국 질렀습니다. 너무 좋고, 사용하기 너무 편하네요. 5 

92 배송이 빠르고 사용이 편리합니다 5 

93 일반 이어폰보다 사용하기 편하고 좋습니다~ 4 

94 배송은 3일 정도 걸렸었어요상품은 깔끔하니 이쁘고 좋아요스마일 배송이라 분실 위험도 없고 완벽했습니다 5 

페이지 1 

1 약 이주일간 사용해보고 작성하는 후기입니다! 먼저 배송은 정말 빨랐어요! 이틀만에 받아서 빠른 배송에 만족했습니다! 에어캡도 잘 감싸져 있어서 안전하게 잘 배송이 왔고요! 2세대 유선충전 제품이다보니 안에 LED 표시가 있습니다. 몇 분들이 1세대 케이스가 아니냐고 많이 물어보시는데 2세대 케이스 맞아요! 정품등록을 하면서 에어팟2임을 확실하게 알 수 있고 정품등록 또한 잘 되었습니다. 구성은 에어팟, 충전 케이스, 라이트닝 케이블 다 있습니다! 참고로 정품등록 할 때 모바일로는 안 되는 것 같아요. 노트북으로 하니까 바로 되더라고요! 정품등록이 안 되시는 분들 중 모바일로 하셨

13 후기에 제품에 대한 하자 얘기가 많아 망설이다 주문했는데 다행이 깨끗한 정상품이 와서 다행이네요디자인도 예쁘고 마감도 깔끔해요무엇보다 음질, 통화품질은 최고예요안드로이드폰이라 사용 가능한 기능에 제약이 있겠지만 통화, 음악, 유튜브가 메인이라 불편함은 그닥 없어요선이 없다는 것이 완전 신세계입니다 5 

14 구매한지 1일만에 배송 완료 되었습니다.배송은 매우 빠르고, 아이폰에 테스트 하였고 아내가 너무 잘 들리고 깨끗한 음질이라고좋아라 합니다. 실물을 봤는데 역시 애플제품이라 그런지 마감도 확실히 다르고디자인도 아주 이쁘네요,많이 파세요. 5 

15 배송 이틀 만에 왔구여 정품 인증 바로 했구여 포장도 깔끔하게 왔어요 하자 하나없이 깔끔하고 좋아요 지인들이 무조건 공홈에서 사라고 해서 조금 불안하긴 했는데 받고나니까 그런거 1도 없고 싸게 잘산거 같다는 생각밖에 안드네요 감사함돠 잘쓸게요~~ 5 

16 에어팟 사용 후 삶의 질이 확 증가했습니다. 확실히 에어팟 사고 난 뒤에는 기존 이어폰 줄은 성가시게 느껴져요. 돈 좀 더 얹더라도 무선 충전이 편한 것 같아요. 그리고 작아서 잘 안 보이고 안 찾아지니 케이스는 쨍한 색의 톡톡 튀는 디자인 추천해용>.< 5 

17 출고는 빨랐으나 택배사에서 하루 지연되서 월요일 출고 수요일에 받았습니다. (한진 믿거)간혹 마감 불량이라든가 기기 결함이 있다고 교환한다는 후기가 보여 불안했습니다만 제껀 외형, 기기작동 모두 괜찮았네요. 솔직히 마감에 허술한 부분은 신경쓰이지 않았지만 기계 자체 결함은 안고 갈 수 없다 생각했거든요..결론적으론 유선 이어폰 사용하다 에어팟 사용하니 신세계입니다. 너무 편해요. 고민하는 분들은 질러도 후회없을 것 같습니당 5 

18 배송진짜 너무 맘에 들었어요 작은 박스에 제주도인데 2일걸려서 직접 수령했네요. 비싼제품이라 걱정했는데 오는시간까지 알려줘서 잘 받았어요. 타사 블루투스 이어폰 써봤는데 이건 정말 가볍고 착용감이 크지 않아서 좋더라구요 디자인도 좋구요. 양쪽 다 음질 좋고

147 깔끔하니 좋네요오오 5 

148 유선 이어폰 쓰다가 잊어버려서 에어팟으로 바꿔보니엄청 편리하고 좋네요.제품 외관도 좋고 소리도 다 잘들리고 좋아요. 정품등록 완료. 잘쓰겠습니다. 5 

149 충천도 빠르고 음질도 좋고. 디자인도 이쁘네요.잘 사용하겠습니다. 사업 더욱더 번창하세요 5 

150 에어팟은 항상 사는거지만 최고네요 품질도 좋고 디자인도 예쁘고,, 이번에 프로 나왓지만 싸게 산다면 에어팟 2도 고려해볼만하다고 생각합니다 프로도 좋고요. 감사합니다 다음에도 또 구매할게요 5 

151 성능 굿 디자인 굿 아이가 좋아해요 둘째아이것도 사려고 해요 5 

152 2주 간 사용해본결과 아주 만족스럽고 상태며 외관이며 다 맘에 들었습니다!^^ 5 

153 오전 주문해서 당일 발송, 바로 다음날에 받았습니다! 포장 및 사은품 만족스럽고, 다행히 뽑기운도 좋은지, 제품 외관이나 성능 (아직까진) 만족스럽습니다. 5 

154 배송이 엄청빠르고 디자인도 너무 예쁘네요 5 

155 깔끔하고 이쁩니당! 5 

156 좋아요. 음질도 좋고 깔끔하니 좋네요~~다음에도 또 이용할게요^^ 5 

157 배송도 하루만에 오고 제품도 너무 깔끔하니 좋아요!짱짱!! 5 

158 모양귀엽고, 음질좋고, 착용감 좋음. 단, 가격대가 비싼듯.. 5 

159 사용하기 편하고 디자인도 이뻐서 상품에 아저 만족합니다. 배송도 매우 빨라서 좋았습니다. 4 

160 디자인도 이쁘고 마음에 듭니다 음향도 좋은거 갔아요 4 

페이지 9 

161 방금 열어서 케이스끼워봤는데 너무이쁘고 좋네요~~ 5 

162 좋아요 잘들리고 예쁘고..만족합니다 5 

163 외관이나 이어폰 모두 깔끔하네요 5 

164 감사합니다 이쁘고 좋아요 5 

165 이쁘고좋아요 삶의질향상 4 

166 이쁘고 좋아요 감사요 4 

167 배송도 빠른데다가! 포장도 깔끔해서 좋았어요! 정품인증도 잘 되서 기분이 좋네요! 감사히 잘 쓰겠습니다^--^!! 5 

168 딸한테 사줬는데 너무 좋아해요 예쁘

4 갤럭시 쓰는데 기존 LG 톤플러스 블투가 망가져서 무상보증기간도 지나고 ㅠㅠ 눈팅만 하던 2세대를 구매했어요. 호환에 대한 걱정은 이미 블로그를 통해 많이 봤고, 초기 세팅은 회사 동료분 아이폰에 등록해서 했네요~ 배터리는 거의 완충상태고 무선 충전도 잘대요~ 영업일해서 하루 종일 끼고 있는데 귀가 안아프네요. 플랜트로닉스것도 써봣는데 3시간 넘음 귀아파서 빼고 있었거든요. 음질은 거의 부드러운 중저음의 느낌? 목소리도 잘 들린다 하고, 총알배송에 주문한 담날 바루 사용중입니다. 넘 맘에 들어요. 악세사리 충동 불어서 케이스랑 키링도 구매하고 ㅎㅎ 만족스러워요~ 5 

5 배송진짜 너무 맘에 들었어요 작은 박스에 제주도인데 2일걸려서 직접 수령했네요. 비싼제품이라 걱정했는데 오는시간까지 알려줘서 잘 받았어요. 타사 블루투스 이어폰 써봤는데 이건 정말 가볍고 착용감이 크지 않아서 좋더라구요 디자인도 좋구요. 양쪽 다 음질 좋고 기기와 연결도 쉽게 ？營윱求？. 많이파새요 5 

6 유선 이어폰 쓰다가 이거 쓰니 완전 신세계네요. 핸폰은 밖에 두고 화장실에서 일 볼 때도 끊김없이 쓸 수 있어요. 이렇게 편할수가 ㅋㅋㅋ 사이즈도 귀에 쏙 들어가서 잘 빠지지도 않고 그렇다고 이물감 때문에 귀가 아프지도 않고. 좋은 가격에 득템. 많이 파십시오-! 5 

7 다른 블루투스 사용하고 있었지만 장시간 사용하면 귀가 아파서 고민하다가 에어팟 구입했습니다
배송도 빠르게 오고 제품도 이상 없이 잘되고 있습니다
다만 부재시 배송완료하고 따로 연락이 없는게 살짝 아쉽네요~ 감사합니다 5 

8 착용감도 좋고 음질도 좋아요 다만 음량 조절 시 1단계의 갭이 커서 많이 아쉬워요 음량 조절을 조금 세분화할 수 있다면 정말 좋겠어요 3 

9 동생이 잘사용하고있습니다. 근데 1시간 이상사용하면 귀가 아파요. 귀모양이 맞는 사람이 있을까요 ..? ㅠㅜ 애플에게 묻고싶다 4 

10 기존에 버즈를썼었는데 그건 계속끼고있으면귀가아프구 상대방이 잘안들린되서 통화품질이너무안좋았었는데 이거완전신세

In [19]:
dataset = pd.read_csv("output.csv", index_col=0)  #csv에서 가져오기
print(dataset)

       type category                                             review  star
1   에어팟 1세대       음질  2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...     5
2   에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
3   에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
4   에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
5   에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
..      ...      ...                                                ...   ...
9   에어팟 2세대      사용성  구하기 힘든 물건 판매해 주시고\n저렴한 판매와 빠른 배송 감사 드립니다.\n1세대...     5
1   에어팟 2세대      조작성  구입일이 아니라 제조일로부터 일정기간 경과된 제품이라 정품등록에 시간이 많이 소요되...     3
2   에어팟 2세대      조작성  칼배송!\n에어팟 처음 쓰는데 사용법도 간단합니다!\n정품 등록하는데 시리얼넘버는 ...     5
3   에어팟 2세대      조작성                                    품질 좋아요 간편해서 좋고.     5
4   에어팟 2세대      조작성  우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...     5

[1243 rows x 4 columns]
